# Converting a TensorFlow model to ONNX using TensorFlow-ONNX (TFONNX)

_Note: This example borrows from a PyTorch sample test used in BentoML_

# Section 1: Create a simple TensorFlow model.

## Section 1.1: Import the packages we need:

In [ ]:
import numpy as np
import tensorflow as tf

## Section 1.2: Build a simple model using the TensorFlow function capability.

In [ ]:
class simplematmul(tf.Module):
    def __init__(self):
        super().__init__()
        self.weights = np.asfarray([[1.0], [1.0], [1.0], [1.0], [1.0]], dtype=np.float32)
        self.dense = lambda inputs: tf.matmul(inputs, self.weights)

    @tf.function(input_signature=[tf.TensorSpec(shape=[1, 5], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        return self.dense(inputs)


In [ ]:
# Create an instance of our model:
model1 = simplematmul()

## Section 1.3: Test our model - it should return a result of 15.

In [ ]:
#  Let's create test data to make sure our model is working
test_data = np.array([[1, 2, 3, 4, 5]], dtype=np.float32)
test_tensor = np.asfarray(test_data)

In [ ]:
print(model1(test_tensor))

## Section 1.4: Save our model. 

In [ ]:
saved_dir = './tmp'
tf.saved_model.save(model1, './tmp',signatures=model1.__call__.get_concrete_function(
    tf.TensorSpec(shape=[1, 5], dtype=tf.float32, name='inputs')))

# Section 2: Convert our simple matrix multiply model using TFONNX

## Section 2.1 Model Conversion with tensorflow-onnx

tensorflow-onnx is an open-source onnx model convertor, available through PYPI and from source here: https://github.com/onnx/tensorflow-onnx 

tensorflow-onnx can convert TensorFlow, TF.Keras and tflite models to the ONNX format.
Both command line and python API conversion is supported. 

The command line tool is much simpler to use for most use cases.

We recommend you always convert your model on the platform you trained it on. (i.e. if trained on Z, convert on Z)

In [ ]:
# Execute tensorflow-onnx command line tool, specifying a target opset of 11 to target for ONNX
!python -m tf2onnx.convert --opset 11 --fold_const --saved-model $saved_dir --output tf_matmul.onnx